# Optimizer Backpropagation
Learning representations by back-propagating errors is a paper by [Rumelhart, Hinton, & Williams (1986)](https://www.nature.com/articles/323533a0), describing how NN may learn by repeatedly adjusting weights.

There are many algorithms that train neural networks, and each have their own benefits and consequences. Modern alogrithms are chosen for the ability to parallelize the vectors, for efficient computation on GPU shaders.

Fig 1: Method comparison, taken from [Sebastian Ruder](http://sebastianruder.com/optimizing-gradient-descent/index.html#visualizationofalgorithms).

![Training Techniques](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/contours_evaluation_optimizers.gif "Training Techniques")

## Classic backpropagation
[Backpropagation](https://en.wikipedia.org/wiki/Backpropagation) works by calculating the weight change $v_i$ for collective weight $\theta_i$ in the neural network:
$$
\theta_t = \theta_{t-1} - v_t
$$
for every iteration $t$. The weight change is calculated depending on the training algorithm. Classic backpropagation calculates a gradient for every weight in the neural network w.r.t. an error function $J$. The gradient is then scaled by a learning rate $\eta$:
$$
v_t = \eta \nabla_{\theta_{t-1}} J(\theta_{t-1}).
$$

The error function in classic backpropagation is the mean squared error (MSE)
$$
J(\theta_t) = \frac{1}{2N} \sum_{i=1}^N \left( \hat{y}_i - y_i \right)^2,
$$
in the $t^\text{th}$ layer, using the prediction $\hat{y}_i$ for expected $y_i$.

### Detailed derivation
Let us consider a general neural network, with $L$ layers. We will use the convention that layer $l=L$ is the output layer, $l=1$ is the first hidden layer, and $l=0$ is the input layer. 

We denote the value of a neuron by $z^l_i$, or across a whole layer as simply the vector $z^l$. The weight between neuron $i$ in layer $l-1$ and neuron $j$ in layer $l$ is represented by the matrix elements $w^l_{ij}$, such that
$$
z^l = W^{l} f \left( z^{l-1} \right),
$$
where $W^l$ is the total weight matrix, and $f$ is the activation function. For ease of notation, we will interchangably use
$$
a^l = f \left( z^l \right).
$$

Using this notation, we can write the action of a dense neural network as the convolution
$$
g(x) := f^{L} \left( W^{L} f^{L-1} \left( W^{L-1} \cdots f^1 \left( W^1 x \right) \cdots \right) \right),
$$
such that a prediction $\hat{y} = g(x)$, with the convention that $x:=a^0$, and $\hat{y} := a^L$.

We are interested in evaluating a cost/error function $E(y, \hat{y})$, where $y$ are the true outputs. We wish to analyse how this cost changes with weights 
$$
\frac{ \partial E}{\partial W^l}.
$$

To do this, we first use chain-rule expansion on the error derivative with respect to the inputs $x$:
$$
\frac{\partial E}{\partial x} = \frac{d C}{d a^L}\frac{d a^L}{d z^L}\frac{d z^L}{d a^{L-1}} \cdots \frac{d a^1}{d z^1} \frac{\partial z^1}{\partial x}.
$$

We note that
$$
\frac{d a^l}{d z^l} = \left( f^l \right) ^\prime =  \left. \frac{\partial f^l}{\partial z^l} \right\vert _{z^l},
$$
and
$$
W^l = \frac{d z^{l}}{d a^{l-1}}.
$$

The total gradient $\nabla$ is the transpose of the above, describing the derivative of the output in terms of the input:
$$
\nabla_x E = \left( W^1 \right)^T \cdot \left( f^1 \right)^{\prime} \cdots \left( W^{L-1} \right)^T \cdot \left( f^{L-1} \right)^{\prime} \cdot \left( W^L \right)^T \cdot \left( f^L \right)^{\prime} \cdot \nabla_{a^L} E.
$$

We introduce the auxiliary quantity $\delta^l$ for the partial product, interpreted as the *error at level $l$* and is a vector with dimension equal to the number of neurons at layer $l$,
$$
\delta^l := \left( f^l \right)^{\prime} \cdot \left( W^{l+1} \right)^T \cdots \left( W^{L-1} \right)^T \cdot \left( f^{L-1} \right)^{\prime} \cdot \left( W^L \right)^T \cdot \left( f^L \right)^{\prime} \cdot \nabla_{a^L} E.
$$

This quantity may be calculated recursively
$$
\delta^{l-1} = \left( f^{l-1} \right)^{\prime} \cdot \left( W^l \right)^T \cdot \delta^l,
$$
and allows us to succinctly express the gradient of $C$ with respect to the weights as
$$
\nabla_{W^l}E = \left( a^{l-1} \right)^T \delta^l.
$$

Backpropagation is then repeatedly applying this equation to determine how the weights should be change to minimize the error function, for each layer from $l=L$ to $l=1$. 

There are a few additional things to note in this approach: backpropagation, as an alogrithm, computes gradients over the whole layers, as opposed to calculating
$$
\frac{\partial E}{\partial w_{ij}^l}
$$
as such a calculation would be computationally inefficient.

It is also worth mentioning that in this notation the biases are encoded as a weight acting on $a^l_0=1$, such that
$$
b_i^l = w_{0i}^k,
$$
and consequently our summations run from 0 instead of 1.

### Learning Rate

The learning rate $\eta$ is an important concept: setting the learning rate too low will converge to a good solution, but take many iterations. If the learning rate is too high, the process may jump out of the minima it is trying to descend into.

Backpropagation is also a type of [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) -- the gradient with respect to each weight provides an indication as how to modify the weight for better convergence, and as such $\eta$ draws a direct analogy to more convention gradient descent methods.

### Worked example: MSE
Let us examine now the case where the error function is the mean squared error (MSE),
$$
E(y, \hat{y}) = \frac{1}{N} \sum_{i=0}^N \left( \hat{y}_i - y_i \right)^2,
$$
summing over all $N$ elements of $y$.

In order to calculate the derivative of $E$ with respect to the weights $W^l$, we only need to calculate $\delta^l$ for each layer, for which we have a recursive formula, starting with
$$
\delta^L = \left( W^L \right) ^T \left( f^L \right) ^\prime \nabla_a^L E
$$
for which we use the fact that the derivative of a sum is the sum of the derivatives, and calculate
$$
\frac{\partial E}{\partial \hat{y_i}} = (\hat{y}_i - y_i).
$$

We then find $\left( f^L \right) ^\prime$; for this, we could choose a [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) function, such that
$$
f \left( z^L \right) = \frac{1}{1+e^{-z^L}},
$$
giving first derivative
$$
\left( f^L \right) ^\prime = \frac{\partial \hat{y}}{\partial z^L} = \frac{e^{z^L}}{\left( e^{z^L} + 1 \right)^2}.
$$

We assemble the results, and determine
$$
\delta^L = \left( W^L \right) ^T \left( \frac{e^{z^L}}{\left( e^{z^L} + 1 \right)^2} \right) (\hat{y}_i - y_i).
$$

We then use our recursive formula to backpropagate to $l=0$, where $a^0 = x$.

## Momentum backpropagation
Momentum backpropagation adds another term to the update value
$$
v_t = \eta \nabla_{\theta_{t-1}} J\left( \theta_{t-1} \right) + \lambda v_{t-1},
$$
with $\lambda$ representing a momentum, scaling the weight change amount -- this adjusts the weight gradient more forcefully, allowing it to escape false minema.

## Batch and online backpropagation
Another problem to face in training NN is how often the weights should be updated; gradients may be calculated on an element-by-element basis for the training set, or summed into batches, such that the weights are updated once per batch. We refer to these differences as

- Online training: update weights based on gradients from a *single training set element*
- Batch training: update weights based on sum gradient over *all training set elements*, i.e. sum the delta vectors and add together for the overall direction for descent.

For batch training, there is the question of batch size -- once can use anywhere from all of the elements to just pairs of the training set.

We introduce the terminology: *step/iteration* for the number of batches processed, and *epoch* for the number of times the training set has been processed.

## Stochastic gradient descent
[Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) (SGD) is a batch training algorithm, with the exception that the batches are constructed from random sets of training elements, leading to irregular convergence. This is desired in the hope that it minimizes the chance of the network converging to a false minima.

SGD can be used with other concepts, such as momentum backpropagation, weight averaging, or e.g. adaptive gradient algorithms (AdaGrad).

## ADAM
[Adaptive Momentum Estimation](https://arxiv.org/abs/1412.6980) (ADAM) is an update to the RMS propagation algorithms; ADAM runs averages of both the gradients and the second moments of the gradients: that is to say it continously adjusts weights and momenta.

The first *moment* is the *mean*, and the second is the *variance*. We denote momentum now $\lambda \rightarrow m$, for consistency with literature. The parameter update is given by for iteration $t$ for first moment
$$
m_w^{t+1} = \beta_1 m_w^t + (1 - \beta_1) \nabla_w J^t
$$
and second moment
$$
v_w^{t+1} = \beta_2 v_w^t + (1 - \beta_2) \left( \nabla_w L^t \right) ^2
$$
To correct for the bias towards zero in the inital training cycles, we correct the moments with
$$
\hat{m}_w = \frac{m_w^{t+1}}{1- \beta_1^{t+1}}
$$
and
$$
\hat{v}_w = \frac{v_w^{t+1}}{1-\beta_2^{t+1}}
$$

The total update is then given by
$$
w^{t+1} = w^{t} - \eta \left( \frac{\hat{m}_w}{\sqrt{\hat{v}_w} + \epsilon} \right)
$$

We use a small value for $\epsilon \sim 10^{-8}$, and Kingma and Ba (2014) propose hyper parameters $\beta_1 = 0.9$ and $\beta_2 = 0.999$.

*Note:* squaring and square-rooting is done element wise.

## Application to TensorFlow
Tensorflow accepts the following update rules
- Adam
- AdaGrad
- [FTRL](https://www.eecs.tufts.edu/~dsculley/papers/ad-click-prediction.pdf)
- Momentum
- RMSProp
- SGD

We can specify them in our model during the model compilation stage: e.g.

In [ ]:
model.compile(
    optimizer='adam'
)